In [2]:
import tweepy

In [3]:
'https://twitter.com/alvarodias_'
'https://twitter.com/cirogomes'
'https://twitter.com/guilhermeboulos'
'https://twitter.com/jairbolsonaro'
'https://twitter.com/geraldoalckmin'
'https://twitter.com/marinasilva'
'https://twitter.com/lulaoficial'
'https://twitter.com/meirelles'

'https://twitter.com/meirelles'

In [6]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

# public_tweets = api.home_timeline()
# for tweet in public_tweets:
#     print(tweet.text)

test?


In [8]:
user = api.get_user('twitter')
print(user.screen_name)
print(user.followers_count)

Twitter
55215320
